In [1]:
import polars as pl
import csv
import glob
import os

In [2]:
def clean_file(input_path, output_path):
    with open(input_path, "r", encoding="utf-8") as infile, open(output_path, "w", encoding="utf-8") as outfile:
        reader = csv.reader(infile, delimiter=";", quotechar='"', escapechar="\\")
        writer = csv.writer(outfile, delimiter=";", quotechar='"', escapechar="\\", quoting=csv.QUOTE_MINIMAL)
        
        for row in reader:
            if row.count('"') % 2 != 0:  
                continue
            max_columns = 24  
            row = row[:max_columns] + [""] * (max_columns - len(row))
            writer.writerow(row)

In [3]:
extract_folder = "extract"
cleaned_folder = "cleaned"

In [4]:
dat_files = glob.glob(f"{extract_folder}/*.DAT")
print(f"Number of .dat files found: {len(dat_files)}")

Number of .dat files found: 5958


In [12]:
os.makedirs(cleaned_folder, exist_ok=True)

dat_files = glob.glob(f"{extract_folder}/*.DAT")

for file in dat_files:
    cleaned_path = os.path.join(cleaned_folder, os.path.basename(file))
    clean_file(file, cleaned_path)

In [13]:
df_list = []
df_schema = {
    'Record Type': pl.Utf8,
    'District Code': pl.Utf8,
    'Property Id.': pl.Utf8,
    'Sale Counter': pl.Utf8,
    'Download Datetime': pl.Utf8,
    'Property Name': pl.Utf8,
    'Property Unit Number': pl.Utf8,
    'Property House Number': pl.Utf8,
    'Property Street Name': pl.Utf8,
    'Property Locality': pl.Utf8,
    'Property Post Code': pl.Utf8,
    'Area': pl.Utf8,
    'Area Type': pl.Utf8,
    'Contract Date': pl.Utf8,
    'Settlement Date': pl.Utf8,
    'Purchase Price': pl.Utf8,
    'Zoning': pl.Utf8,
    'Nature of Property': pl.Utf8,
    'Primary Purpose': pl.Utf8,
    'Strata Lot Number': pl.Utf8,
    'Component code': pl.Utf8,
    'Sale Code': pl.Utf8,
    '% Interest of Sale': pl.Utf8,
    'Dealing Number': pl.Utf8
}

In [15]:
dat_cleaned = glob.glob(f"{cleaned_folder}/*.DAT")
for file in dat_cleaned:
    try:
        df = pl.read_csv(
            file,
            separator=";",
            has_header=False,
            schema = df_schema,
            truncate_ragged_lines=True,
            ignore_errors=True
        ).with_columns(
            [pl.col(column).cast(pl.Utf8) for column in df_schema.keys()])
        df_list.append(df)
    except Exception as e:
        print(f"Error reading {file}: {e}")

In [16]:
if df_list:
    try:
        df_combined = pl.concat(df_list)
        df_filtered = df_combined.filter(pl.col("Record Type") == "B")
        df_filtered.write_csv("filtered_data.csv")
        print("Filtered data saved to 'filtered_data.csv'.")
    except Exception as e:
        print(f"Error during concatenation or filtering: {e}")
else:
    print("No valid DataFrames to process.")

Filtered data saved to 'filtered_data.csv'.


In [17]:
df_filtered.head()

Record Type,District Code,Property Id.,Sale Counter,Download Datetime,Property Name,Property Unit Number,Property House Number,Property Street Name,Property Locality,Property Post Code,Area,Area Type,Contract Date,Settlement Date,Purchase Price,Zoning,Nature of Property,Primary Purpose,Strata Lot Number,Component code,Sale Code,% Interest of Sale,Dealing Number
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""B""","""708""","""2021430""","""1""","""20240415 01:08""",null,"""26""","""152""","""BULWARA RD""","""PYRMONT""","""2009""",null,null,"""20240226""","""20240408""","""1028000""",null,"""R""","""RESIDENCE""","""26""",null,null,"""0""","""AT964426"""
"""B""","""708""","""2035255""","""2""","""20240415 01:08""",null,"""1208""","""61""","""MACQUARIE ST""","""SYDNEY""","""2000""","""221""","""M""","""20240113""","""20240405""","""6750000""",null,"""R""","""RESIDENCE""","""116""",null,null,"""0""","""AT960675"""
"""B""","""708""","""2025098""","""3""","""20240415 01:08""",null,"""202""","""6 E""","""COWPER WHARF RDWY""","""WOOLLOOMOOLOO""","""2011""",null,null,"""20240320""","""20240405""","""1200000""",null,"""R""","""RESIDENCE""","""43""",null,null,"""0""","""AT959987"""
"""B""","""708""","""2046244""","""4""","""20240415 01:08""",null,"""533""","""99""","""JONES ST""","""ULTIMO""","""2007""",null,null,"""20240226""","""20240408""","""1335000""",null,"""R""","""RESIDENCE""","""218""",null,null,"""0""","""AT963711"""
"""B""","""708""","""2046582""","""5""","""20240415 01:08""",null,"""17""","""330""","""WATTLE ST""","""ULTIMO""","""2007""",null,null,"""20240301""","""20240405""","""300000""",null,"""3""","""COMMERCIAL""","""17""",null,null,"""0""","""AT960874"""


In [18]:
df_cleaning = df_filtered.with_columns(
    pl.col("Contract Date").str.strptime(pl.Date, "%Y%m%d", strict=False).alias("Contract Date")
)

In [19]:
df_cleaning = df_cleaning.select('District Code',
                            'Property Id.',
                            'Property Name',
                            'Property Unit Number',
                            'Property House Number',
                            'Property Street Name',
                            'Property Locality',
                            'Property Post Code',
                            'Area',
                            'Area Type',
                            'Contract Date',
                            'Settlement Date',
                            'Purchase Price',
                            'Primary Purpose',
                            'Strata Lot Number')

In [20]:
df_cleaning = df_cleaning.with_columns(pl.col("Area").cast(pl.Float64).alias("Area"))

In [21]:
df_cleaning = df_cleaning.with_columns(
    pl.when(pl.col('Area Type') == 'M')
     .then(pl.col('Area').cast(pl.Float64))
     .when(pl.col('Area Type') == 'H')
     .then(pl.col('Area').cast(pl.Float64) * 10000)
     .otherwise(None)
     .alias('Area (m)')
)

In [22]:
df_cleaning = df_cleaning.with_columns(pl.col("Purchase Price").cast(pl.Int64).alias("Purchase Price"))

In [23]:
df_cleaning = df_cleaning.with_columns(pl.col("Property Id.").cast(pl.Int64).alias("Property Id."))

In [24]:
df_cleaning = df_cleaning.with_columns(
    pl.col("Settlement Date").str.strptime(pl.Date, "%Y%m%d", strict=False).alias("Settlement Date")
)

In [25]:
df_cleaning = df_cleaning.with_columns([
    pl.col('Settlement Date').dt.year().alias('Settlement Year'),
    pl.col('Settlement Date').dt.month().alias('Settlement Month')
])


In [26]:
df_cleaning = df_cleaning.filter(
    pl.col('Settlement Year') >= 2021
)

In [27]:
df_cleaning = df_cleaning.with_columns((pl.col("Purchase Price") / pl.col("Area (m)")).alias("Price per m^2").round(2))

In [28]:
df_cleaning.head()

District Code,Property Id.,Property Name,Property Unit Number,Property House Number,Property Street Name,Property Locality,Property Post Code,Area,Area Type,Contract Date,Settlement Date,Purchase Price,Primary Purpose,Strata Lot Number,Area (m),Settlement Year,Settlement Month,Price per m^2
str,i64,str,str,str,str,str,str,f64,str,date,date,i64,str,str,f64,i32,i8,f64
"""708""",2021430,null,"""26""","""152""","""BULWARA RD""","""PYRMONT""","""2009""",null,null,2024-02-26,2024-04-08,1028000,"""RESIDENCE""","""26""",null,2024,4,null
"""708""",2035255,null,"""1208""","""61""","""MACQUARIE ST""","""SYDNEY""","""2000""",221.0,"""M""",2024-01-13,2024-04-05,6750000,"""RESIDENCE""","""116""",221.0,2024,4,30542.99
"""708""",2025098,null,"""202""","""6 E""","""COWPER WHARF RDWY""","""WOOLLOOMOOLOO""","""2011""",null,null,2024-03-20,2024-04-05,1200000,"""RESIDENCE""","""43""",null,2024,4,null
"""708""",2046244,null,"""533""","""99""","""JONES ST""","""ULTIMO""","""2007""",null,null,2024-02-26,2024-04-08,1335000,"""RESIDENCE""","""218""",null,2024,4,null
"""708""",2046582,null,"""17""","""330""","""WATTLE ST""","""ULTIMO""","""2007""",null,null,2024-03-01,2024-04-05,300000,"""COMMERCIAL""","""17""",null,2024,4,null


In [29]:
df_cleaning.n_unique

<bound method DataFrame.n_unique of shape: (192_702, 19)
┌───────────┬───────────┬───────────┬───────────┬───┬──────────┬───────────┬───────────┬───────────┐
│ District  ┆ Property  ┆ Property  ┆ Property  ┆ … ┆ Area (m) ┆ Settlemen ┆ Settlemen ┆ Price per │
│ Code      ┆ Id.       ┆ Name      ┆ Unit      ┆   ┆ ---      ┆ t Year    ┆ t Month   ┆ m^2       │
│ ---       ┆ ---       ┆ ---       ┆ Number    ┆   ┆ f64      ┆ ---       ┆ ---       ┆ ---       │
│ str       ┆ i64       ┆ str       ┆ ---       ┆   ┆          ┆ i32       ┆ i8        ┆ f64       │
│           ┆           ┆           ┆ str       ┆   ┆          ┆           ┆           ┆           │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪══════════╪═══════════╪═══════════╪═══════════╡
│ 708       ┆ 2021430   ┆ null      ┆ 26        ┆ … ┆ null     ┆ 2024      ┆ 4         ┆ null      │
│ 708       ┆ 2035255   ┆ null      ┆ 1208      ┆ … ┆ 221.0    ┆ 2024      ┆ 4         ┆ 30542.99  │
│ 708       ┆ 2025098   ┆ null    

In [30]:
unit_df = df_cleaning.filter((pl.col("Property Unit Number") != "null") | (pl.col("Strata Lot Number") != "null"))

In [31]:
unit_df.head()

District Code,Property Id.,Property Name,Property Unit Number,Property House Number,Property Street Name,Property Locality,Property Post Code,Area,Area Type,Contract Date,Settlement Date,Purchase Price,Primary Purpose,Strata Lot Number,Area (m),Settlement Year,Settlement Month,Price per m^2
str,i64,str,str,str,str,str,str,f64,str,date,date,i64,str,str,f64,i32,i8,f64
"""708""",2021430,null,"""26""","""152""","""BULWARA RD""","""PYRMONT""","""2009""",null,null,2024-02-26,2024-04-08,1028000,"""RESIDENCE""","""26""",null,2024,4,null
"""708""",2035255,null,"""1208""","""61""","""MACQUARIE ST""","""SYDNEY""","""2000""",221.0,"""M""",2024-01-13,2024-04-05,6750000,"""RESIDENCE""","""116""",221.0,2024,4,30542.99
"""708""",2025098,null,"""202""","""6 E""","""COWPER WHARF RDWY""","""WOOLLOOMOOLOO""","""2011""",null,null,2024-03-20,2024-04-05,1200000,"""RESIDENCE""","""43""",null,2024,4,null
"""708""",2046244,null,"""533""","""99""","""JONES ST""","""ULTIMO""","""2007""",null,null,2024-02-26,2024-04-08,1335000,"""RESIDENCE""","""218""",null,2024,4,null
"""708""",2046582,null,"""17""","""330""","""WATTLE ST""","""ULTIMO""","""2007""",null,null,2024-03-01,2024-04-05,300000,"""COMMERCIAL""","""17""",null,2024,4,null


In [32]:
unit_df_filter = unit_df.filter(pl.col("Primary Purpose").is_in(["UNIT", "RESIDENCE", "HOUSE UNIT"]))

In [33]:
unit_df_filter.head()

District Code,Property Id.,Property Name,Property Unit Number,Property House Number,Property Street Name,Property Locality,Property Post Code,Area,Area Type,Contract Date,Settlement Date,Purchase Price,Primary Purpose,Strata Lot Number,Area (m),Settlement Year,Settlement Month,Price per m^2
str,i64,str,str,str,str,str,str,f64,str,date,date,i64,str,str,f64,i32,i8,f64
"""708""",2021430,null,"""26""","""152""","""BULWARA RD""","""PYRMONT""","""2009""",null,null,2024-02-26,2024-04-08,1028000,"""RESIDENCE""","""26""",null,2024,4,null
"""708""",2035255,null,"""1208""","""61""","""MACQUARIE ST""","""SYDNEY""","""2000""",221.0,"""M""",2024-01-13,2024-04-05,6750000,"""RESIDENCE""","""116""",221.0,2024,4,30542.99
"""708""",2025098,null,"""202""","""6 E""","""COWPER WHARF RDWY""","""WOOLLOOMOOLOO""","""2011""",null,null,2024-03-20,2024-04-05,1200000,"""RESIDENCE""","""43""",null,2024,4,null
"""708""",2046244,null,"""533""","""99""","""JONES ST""","""ULTIMO""","""2007""",null,null,2024-02-26,2024-04-08,1335000,"""RESIDENCE""","""218""",null,2024,4,null
"""708""",2046582,null,"""108""","""330""","""WATTLE ST""","""ULTIMO""","""2007""",null,null,2024-02-26,2024-04-08,415800,"""RESIDENCE""","""108""",null,2024,4,null


In [34]:
output_file2 = "./units.csv"

unit_df_filter.write_csv(output_file2)

In [35]:
unit_groupby = unit_df_filter.group_by(['Property Locality', 'Settlement Year']).agg(pl.col('Price per m^2').mean().alias('Average Price per m^2 per year').round(2))
unit_groupby.sort(['Property Locality', 'Settlement Year'])

Property Locality,Settlement Year,Average Price per m^2 per year
str,i32,f64
"""ABBOTSFORD""",2024,9810.86
"""ABERDARE""",2024,2511.52
"""ABERDEEN""",2024,1968.34
"""ABERGLASSLYN""",2024,2241.29
"""ABERMAIN""",2024,2112.83
…,…,…
"""YOOGALI""",2024,1248.15
"""YOUNG""",2024,1109.89
"""YOWIE BAY""",2024,5393.94


In [36]:
output_file3 = "./units_grouped.csv"

unit_groupby.write_csv(output_file3)

In [37]:
house_df = df_cleaning.filter(
    (pl.col("Property Unit Number").is_null() & pl.col("Strata Lot Number").is_null()))

In [38]:
house_df.head()

District Code,Property Id.,Property Name,Property Unit Number,Property House Number,Property Street Name,Property Locality,Property Post Code,Area,Area Type,Contract Date,Settlement Date,Purchase Price,Primary Purpose,Strata Lot Number,Area (m),Settlement Year,Settlement Month,Price per m^2
str,i64,str,str,str,str,str,str,f64,str,date,date,i64,str,str,f64,i32,i8,f64
"""708""",2106214,null,null,"""106""","""ALBION ST""","""SURRY HILLS""","""2010""",75.9,"""M""",2024-02-24,2024-04-10,2600000,"""RESIDENCE""",null,75.9,2024,4,34255.6
"""708""",2121933,null,null,"""14""","""EGAN ST""","""NEWTOWN""","""2042""",107.5,"""M""",2024-03-08,2024-04-05,1700000,"""RESIDENCE""",null,107.5,2024,4,15813.95
"""708""",2108959,null,null,"""36""","""BINNING ST""","""ERSKINEVILLE""","""2043""",116.13,"""M""",2024-03-02,2024-04-08,2510000,"""RESIDENCE""",null,116.13,2024,4,21613.71
"""708""",2114073,null,null,"""33""","""CAMPBELL ST""","""NEWTOWN""","""2042""",75.9,"""M""",2024-02-24,2024-04-05,1365000,"""RESIDENCE""",null,75.9,2024,4,17984.19
"""708""",2110506,null,null,"""505""","""BOURKE ST""","""SURRY HILLS""","""2010""",132.8,"""M""",2024-02-28,2024-04-10,3100000,"""RESIDENCE""",null,132.8,2024,4,23343.37


In [39]:
house_df_filter = house_df.filter(pl.col("Primary Purpose").is_in(["RESIDENCE", "DWELLING", "HOUSE AND FARM"]))

In [40]:
house_df_filter.head()

District Code,Property Id.,Property Name,Property Unit Number,Property House Number,Property Street Name,Property Locality,Property Post Code,Area,Area Type,Contract Date,Settlement Date,Purchase Price,Primary Purpose,Strata Lot Number,Area (m),Settlement Year,Settlement Month,Price per m^2
str,i64,str,str,str,str,str,str,f64,str,date,date,i64,str,str,f64,i32,i8,f64
"""708""",2106214,null,null,"""106""","""ALBION ST""","""SURRY HILLS""","""2010""",75.9,"""M""",2024-02-24,2024-04-10,2600000,"""RESIDENCE""",null,75.9,2024,4,34255.6
"""708""",2121933,null,null,"""14""","""EGAN ST""","""NEWTOWN""","""2042""",107.5,"""M""",2024-03-08,2024-04-05,1700000,"""RESIDENCE""",null,107.5,2024,4,15813.95
"""708""",2108959,null,null,"""36""","""BINNING ST""","""ERSKINEVILLE""","""2043""",116.13,"""M""",2024-03-02,2024-04-08,2510000,"""RESIDENCE""",null,116.13,2024,4,21613.71
"""708""",2114073,null,null,"""33""","""CAMPBELL ST""","""NEWTOWN""","""2042""",75.9,"""M""",2024-02-24,2024-04-05,1365000,"""RESIDENCE""",null,75.9,2024,4,17984.19
"""708""",2110506,null,null,"""505""","""BOURKE ST""","""SURRY HILLS""","""2010""",132.8,"""M""",2024-02-28,2024-04-10,3100000,"""RESIDENCE""",null,132.8,2024,4,23343.37


In [41]:
output_file4 = "./house.csv"

house_df_filter.write_csv(output_file4)

In [42]:
house_groupby = house_df_filter.group_by(['Property Locality', 'Settlement Year']).agg(pl.col('Price per m^2').mean().alias('Average Price per m^2 per year').round(2))
house_groupby.sort(['Property Locality', 'Settlement Year'])


Property Locality,Settlement Year,Average Price per m^2 per year
str,i32,f64
"""AARONS PASS""",2024,1.0
"""ABBOTSBURY""",2024,2174.99
"""ABBOTSFORD""",2024,8633.88
"""ABERCROMBIE""",2024,909.46
"""ABERCROMBIE RIVER""",2024,0.44
…,…,…
"""YOUNG""",2024,391.67
"""YOWIE BAY""",2024,3338.23
"""YOWRIE""",2024,6.73


In [43]:
house_groupby.head()

Property Locality,Settlement Year,Average Price per m^2 per year
str,i32,f64
"""WARATAH""",2023,1683.67
"""KIAMA""",2024,3255.04
"""LAKE CARGELLIGO""",2024,162.28
"""TREGEAGLE""",2024,29.64
"""MEDOWIE""",2024,946.4


In [44]:
output_file5 = "./house_grouped.csv"

house_groupby.write_csv(output_file5)